In [1]:
import Robogame as rg
import networkx as nx
import altair as alt
import time, json
import pandas as pd
import numpy as np
import nx_altair as nxa

In [2]:
# load the real game data
directory = 2   # change this to load a different sample


directory = str(directory)

robotdata = pd.read_csv("../server/example"+directory+"/examplematch"+directory+".robotdata.csv")


def read_tree_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return nx.tree_graph(js_graph)

def read_graph_file(filename):
    with open(filename) as f:
        js_graph = json.load(f)
    return nx.node_link_graph(js_graph)

robottree = read_tree_file("../server/example"+directory+"/examplematch"+directory+".tree.json")
robotsocial = read_graph_file("../server/example"+directory+"/examplematch"+directory+".socialnet.json")


In [3]:
# let's look at some basic stats
prod = robotdata[robotdata.Productivity > 0]
unprod = robotdata[robotdata.Productivity < 0]

print("Number of productive robots: ",len(prod))
print("Productivity if we got them all: ",prod.Productivity.sum())
print("---")
print("Number of productive robots: ",len(unprod))
print("Productivity if we got them all: ",unprod.Productivity.sum())
print("---")
print("Productivity if we got all the robots: ",robotdata.Productivity.sum())

Number of productive robots:  56
Productivity if we got them all:  2375.585430668706
---
Number of productive robots:  44
Productivity if we got them all:  -1529.5889047250703
---
Productivity if we got all the robots:  845.9965259436356


In [4]:
# let's look at when the robots expired
alt.Chart(robotdata).mark_bar().encode(
    x=alt.X('expires:Q',bin=True),
    y='count()'
)

# for the example3 file we can see that there are more expirations at the end (but that's not always the case)

alt.Chart(...)

In [5]:
# in some games it may be helpful to know if productive robots come
# at the start or end, so let's look at scatter plot
# let's look at when the robots expired
alt.Chart(robotdata).mark_point().encode(
    x=alt.X('expires:Q'),
    y=alt.Y('Productivity:Q')
)

# in the example2 file we see a little bit of a pattern. Robots with more extreme productivity come 
# later in the game (this is true for most "easy" rounds)

alt.Chart(...)

In [6]:
# we can also start to look at correlations between productivity and various properties
# depending on the match, we'll see more or less
quantProps = ['Astrogation Buffer Length','InfoCore Size',
          'AutoTerrain Tread Count','Polarity Sinks',
          'Cranial Uplink Bandwidth','Repulsorlift Motor HP',
          'Sonoreceptors']
nomProps = ['Arakyd Vocabulator Model','Axial Piston Model','Nanochip Model']

c1 = alt.Chart(robotdata).mark_point().encode(
    x='Productivity:Q',
    y='AutoTerrain Tread Count:Q'
)

c2 = alt.Chart(robotdata).mark_point().encode(
    x='Productivity:Q',
    y='Cranial Uplink Bandwidth:Q'
)

c3 = alt.Chart(robotdata).mark_point().encode(
    x='Productivity:Q',
    y='Arakyd Vocabulator Model:N'
)

c1&c2&c3

# if we can figure this out, we might be able to tell which robots are likely to have better parts

alt.VConcatChart(...)

In [7]:
def getTimeSeries(robotid):
    # let's look at a timeseries for one of the robots
    somerobot = robotdata.loc[[robotid]]

    #print(somerobot.t_1)
    #print(somerobot.expires)
    times = []

    for tm in np.arange(1,101):
        times.append("t_"+str(tm))

    # times holds the column names for the timeseries from the robot file
    somerobot_vals = somerobot[times].melt()
    

    somerobot_vals['expires'] = 'No'
    somerobot_vals.at[int(somerobot.expires),'expires']='Yes'
    
    somerobot_vals = somerobot_vals.reset_index()
    
    ts = alt.Chart(somerobot_vals).mark_line().encode(
        x='index:Q',
        y='value:Q'
    )
    
    temp2 = somerobot_vals[somerobot_vals.expires == 'Yes']
    rule = alt.Chart(temp2).mark_rule(color='red').encode(
            x='index:Q'
    )

    return (ts+rule).properties(
        height=40,
        width=300
    )



In [8]:
getTimeSeries(8)

alt.LayerChart(...)

In [16]:
# let's look at the tree a little
# Compute positions for viz.

# first, the json file "forgets" things like labels/ids/etc. so we 
# want to add a property explicitly that has this information

for n in np.arange(0,len(robottree.nodes)):
    # for every node, set its label property to string of the numerical id
    robottree.nodes[n]['label'] = str(n)
    
pos = nx.nx_agraph.graphviz_layout(robottree, prog="twopi")


# let's break it apart into edges and nodes
edgeviz = nxa.draw_networkx_edges(robottree, pos=pos)
nodeviz = nxa.draw_networkx_nodes(robottree, pos=pos)


# add tooltips to the nodes
nodeviz = nodeviz.mark_circle(size=40).encode(tooltip='label:N')
edgeviz + nodeviz


alt.LayerChart(...)

In [10]:
# We can also explore if there are other interesting properties about the data.
# For example, we know that sometimes productivity or parts are localized to specific points in 
# the tree. To see this, we'll have to augment the data a bit.

for n in np.arange(0,100):
    robottree.nodes[n]['Productivity'] = robotdata.loc[n].Productivity

In [11]:
# Draw the graph using Altair, coloring by productivity
nxa.draw_networkx(robottree, pos=pos, node_color='Productivity',node_tooltip=['label:N','Productivity:Q'])

# we can see (for example 3, at least) that there is some "locality" to productive and unproductive robots

alt.LayerChart(...)

In [12]:
# we also believe that robots have similar time series if they are close
# let's check this for two robots in example2 that are close and two that are far

getTimeSeries(19)&getTimeSeries(89)&getTimeSeries(92)

# you can see that the first two are very similar but they are both different from the last

alt.VConcatChart(...)

In [13]:
# let's look at the social network
# Compute positions for viz.
pos = nx.spring_layout(robotsocial)

# Draw the graph using Altair
viz = nxa.draw_networkx(robotsocial, pos=pos,node_tooltip='label:N')

viz

# this particular network is somewhat messy but there is still a core. We might want to prioritize 
# well connected nodes that expire early

alt.LayerChart(...)